In [89]:
import requests
import pandas as pd
from datetime import datetime, timezone, timedelta, date
from OddsJamClient import OddsJamClient
from sqlalchemy import create_engine
from sqlalchemy.exc import IntegrityError
import sqlite3
api_key_path = 'api_key.txt'

def load_api_key(path):
    with open(path, 'r') as file:
        return file.readline().strip()

api_key = load_api_key(api_key_path)

# Convert American odds to decimal odds
def american_to_decimal(odds):
    return (odds / 100) + 1 if odds > 0 else (100 / abs(odds)) + 1

# Calculate implied probability from decimal odds
def implied_probability(decimal_odds):
    return 1 / decimal_odds

# Adjust implied probability to account for the vig
def adjust_for_vig(implied_prob, vig_reduction=0.02):
    return implied_prob / (1 + vig_reduction)

# Calculate expected value (EV)
def calculate_ev(true_prob, odds, stake=100):
    decimal_odds = american_to_decimal(odds)
    profit_if_win = (decimal_odds - 1) * stake
    loss_prob = 1 - true_prob
    ev = (true_prob * profit_if_win) - (loss_prob * stake)
    return ev

# Fetch sports markets dynamically for a specific sport and league
def fetch_sports_markets(api_key, sport, league, sportsbook=['Pinnacle']):
    url = "https://api.opticodds.com/api/v3/markets"
    params = {
        'sport': sport,
        'league': league,
        'key': api_key
    }
    if sportsbook:
        params['sportsbook'] = sportsbook
    headers = {'Authorization': f'Bearer {api_key}'}
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data['data'])
    else:
        print(f"Failed to fetch markets: {response.status_code} - {response.text}")
        return pd.DataFrame()

# Get today's game IDs dynamically
def get_todays_game_ids(api_key, league):
    Client = OddsJamClient(api_key)
    Client.UseV2()
    GamesResponse = Client.GetGames(league=league)
    Games = GamesResponse.Games
    games_data = [{'game_id': game.id, 'start_date': game.start_date} for game in Games]
    games_df = pd.DataFrame(games_data)
    games_df['start_date'] = pd.to_datetime(games_df['start_date'])
    return games_df['game_id'].tolist()

# Fetch game data dynamically and filter based on player or game markets
def fetch_game_data(game_ids, api_key, market_type='game', sport='baseball', league='MLB', sportsbooks=['Pinnacle', 'Caesars'], include_player_name=True):
    """
    Fetch game data dynamically based on the market type (player or game).
    
    :param game_ids: List of game IDs to fetch odds for
    :param api_key: API key for OddsJam
    :param market_type: 'player' for player markets, 'game' for game markets
    :param sport: The sport to fetch markets for
    :param league: The league to fetch markets for
    :param sportsbooks: List of sportsbooks to fetch data from
    :param include_player_name: Whether to include player names (for player markets)
    :return: DataFrame with the fetched game data
    """
    
    # Fetch all markets for the sport and league
    markets_df = fetch_sports_markets(api_key, sport, league, sportsbooks)

    # Separate player and game markets based on market name
    if market_type == 'player':
        markets = markets_df[markets_df['name'].str.contains('Player', case=False)]['name'].tolist()
    elif market_type == 'game':
        markets = markets_df[~markets_df['name'].str.contains('Player', case=False)]['name'].tolist()
    else:
        print(f"Unknown market type: {market_type}")
        return pd.DataFrame()

    url = "https://api-external.oddsjam.com/api/v2/game-odds"
    all_data = []
    
    # Fetch data in chunks for each sportsbook and game ID
    for chunk in [game_ids[i:i + 5] for i in range(0, len(game_ids), 5)]:
        for sportsbook in sportsbooks:
            params = {
                'key': api_key,
                'sportsbook': sportsbook,
                'game_id': chunk,
                'market_name': markets
            }
            response = requests.get(url, params=params)
            if response.status_code == 200:
                data = response.json().get('data', [])
                all_data.extend(data)
            else:
                print(f"Error fetching data: {response.status_code} - {response.text}")
    
    # Convert the fetched data into a dataframe
    rows = []
    for game_data in all_data:
        home_team = game_data.get('home_team', 'Unknown')
        away_team = game_data.get('away_team', 'Unknown')
        odds_list = game_data.get('odds', [])
        
        for item in odds_list:
            row = {
                'Game ID': game_data.get('id', 'Unknown'),
                "Game Name": f"{home_team} vs {away_team}",
                "Bet Name": item.get('name', None),
                'Market Name': item.get('market_name', ''),
                'Sportsbook': item.get('sports_book_name', sportsbook),
                'line': item.get('bet_points', None),
                'Odds': item.get('price', None),
            }
            if include_player_name and market_type == 'player':
                row['Player Name'] = item.get('selection', 'Unknown')
            rows.append(row)
    
    return pd.DataFrame(rows)

# Find plus EV bets by merging sportsbook data and calculating EV
def find_plus_ev_bets(df, threshold=5):
    caesars_df = df[df['Sportsbook'] == 'Caesars']
    pinnacle_df = df[df['Sportsbook'] == 'Pinnacle']
    betmgm_df = df[df['Sportsbook'] == 'BetMGM'].rename(columns={'Odds': 'Odds_betmgm'})
    
    merged_df = pd.merge(caesars_df, pinnacle_df, on=['Game ID', 'Bet Name', 'Market Name'], suffixes=('_caesars', '_pinnacle'))
    merged_df = pd.merge(merged_df, betmgm_df, on=['Game ID', 'Bet Name', 'Market Name'], how='left')
    
    merged_df['decimal_odds_caesars'] = merged_df['Odds_caesars'].apply(american_to_decimal)
    merged_df['decimal_odds_pinnacle'] = merged_df['Odds_pinnacle'].apply(american_to_decimal)
    merged_df['decimal_odds_betmgm'] = merged_df['Odds_betmgm'].apply(american_to_decimal)
    
    merged_df['implied_prob_caesars'] = merged_df['decimal_odds_caesars'].apply(implied_probability)
    merged_df['implied_prob_pinnacle'] = merged_df['decimal_odds_pinnacle'].apply(implied_probability)
    
    merged_df['true_prob_pinnacle'] = merged_df['implied_prob_pinnacle'].apply(adjust_for_vig)
    
    merged_df['EV_caesars'] = merged_df.apply(lambda row: calculate_ev(row['true_prob_pinnacle'], row['Odds_caesars']), axis=1)
    merged_df['EV_betmgm'] = merged_df.apply(lambda row: calculate_ev(row['true_prob_pinnacle'], row['Odds_betmgm']), axis=1)
    
    positive_ev_bets = merged_df[(merged_df['EV_caesars'] > threshold) | (merged_df['EV_betmgm'] > threshold)]
    return positive_ev_bets
import sqlite3

# Function to remove duplicates by checking existing records in the SQL table using sqlite3
def remove_duplicates(df, league, market_type, conn, table_prefix='betting_data'):
    """
    Remove duplicates from the DataFrame by checking existing records in the SQL table.

    :param df: DataFrame to check for duplicates
    :param league: The league name (e.g., 'MLB', 'NFL', 'NCAAF')
    :param market_type: 'player' for player markets, 'game' for game markets
    :param conn: sqlite3 connection to the database
    :param table_prefix: The prefix for the table name
    :return: DataFrame with duplicates removed
    """
    # Determine the table name
    table_name = f"{table_prefix}_{league}_{market_type.capitalize()}"
    
    # Query existing unique IDs from the table (if the table exists)
    cursor = conn.cursor()
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
    table_exists = cursor.fetchone()
    
    if table_exists:
        # Query existing unique IDs from the table
        query = f"SELECT unique_id FROM {table_name}"
        existing_unique_ids = pd.read_sql(query, conn)['unique_id'].tolist()
        
        # Remove rows from the DataFrame where the unique_id already exists in the table
        df = df[~df['unique_id'].isin(existing_unique_ids)]
    
    return df

# Main function to save data and avoid duplicates using sqlite3
def save_dataframe_to_sql(df, league, market_type, conn):
    """
    Save DataFrame to SQL while avoiding duplicates.

    :param df: The DataFrame to save
    :param league: The league (e.g., 'MLB', 'NFL', 'NCAAF')
    :param market_type: The market type ('player' or 'game')
    :param conn: sqlite3 connection to the database
    """
    # Add unique identifier to prevent duplicates
    df['unique_id'] = df['Game ID'].astype(str) + "_" + df['Bet Name'].astype(str)
    
    # Remove duplicates by checking existing records
    df = remove_duplicates(df, league, market_type, conn)
    
    # Save the cleaned DataFrame to SQL
    save_to_sql(df, league, market_type, conn)



# Function to check for missing columns and add them to the SQL table
def ensure_sql_table_columns(conn, df, table_name):
    """
    Ensure the SQL table has all the columns from the DataFrame.
    
    :param conn: sqlite3 connection to the database
    :param df: DataFrame with the columns
    :param table_name: The name of the SQL table
    """
    cursor = conn.cursor()

    # Get the existing columns in the table (if the table exists)
    cursor.execute(f"PRAGMA table_info({table_name})")
    existing_columns = [info[1] for info in cursor.fetchall()]  # Column names are in the second position

    # Get the DataFrame columns
    df_columns = df.columns.tolist()

    # Find missing columns
    missing_columns = [col for col in df_columns if col not in existing_columns]

    # Dynamically add missing columns to the SQL table
    for col in missing_columns:
        # Determine the column type for SQLite (assuming TEXT for simplicity; you can adjust as needed)
        column_type = "TEXT"
        if df[col].dtype in ['int64', 'float64']:
            column_type = "REAL"
        alter_table_query = f"ALTER TABLE {table_name} ADD COLUMN '{col}' {column_type}"
        try:
            cursor.execute(alter_table_query)
            print(f"Added column '{col}' to table '{table_name}'")
        except sqlite3.OperationalError as e:
            print(f"Error adding column '{col}': {e}")

    conn.commit()

# Function to save the DataFrame to a SQL database using sqlite3
def save_to_sql(df, league, conn, table_prefix='betting_data'):
    """
    Save the DataFrame to a SQL database. Avoids duplicates by checking for existing records.
    
    :param df: DataFrame to save
    :param league: The league name (e.g., 'MLB', 'NFL', 'NCAAF')
    :param market_type: 'player' for player markets, 'game' for game markets
    :param conn: sqlite3 connection to the database
    :param table_prefix: The prefix for the table name, followed by league and market type
    """
    # Determine the table name based on the league and market type
    table_name = f"{table_prefix}_{league}"

    # Ensure the table has all the columns from the DataFrame
    ensure_sql_table_columns(conn, df, table_name)

    # Add a unique identifier to avoid duplicates (e.g., based on 'Game ID' and 'Bet Name')
    df['unique_id'] = df['Game ID'].astype(str) + "_" + df['Bet Name'].astype(str)

    # Try inserting the data into the table while avoiding duplicates
    try:
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"Data saved to table {table_name}.")
    except Exception as e:
        print(f"Error saving data: {e}")


# Example usage for fetching both player and game markets

sport = input('sport: ')
league = input('league: ')

# Fetch game IDs for today
game_ids = get_todays_game_ids(api_key, league)

# Fetch player markets dynamically based on the sport/league
player_props_df = fetch_game_data(game_ids, api_key, market_type='player', sport=sport, league=league, sportsbooks=['Pinnacle', 'Caesars'])
player_ev_bets = find_plus_ev_bets(player_props_df, threshold=5)

# Fetch game markets dynamically based on the sport/league
game_props_df = fetch_game_data(game_ids, api_key, market_type='game', sport=sport, league=league, sportsbooks=['Pinnacle', 'Caesars'])
game_ev_bets = find_plus_ev_bets(game_props_df, threshold=5)

# Combine results
final_ev_df = pd.concat([player_ev_bets, game_ev_bets], ignore_index=True)

if __name__ == "__main__":
    conn = sqlite3.connect('betting_data.db')
    if league == "England - Premier League":
        league = 'EPL'
    save_to_sql(final_ev_df, league, conn)
    
    # Close the SQLite connection
    conn.close()



sport: football
league: NCAAF


/var/folders/zf/xwmjxb7n5nvfjg8yckhqkm380000gn/T/ipykernel_82152/1803869653.py:64: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  games_df['start_date'] = pd.to_datetime(games_df['start_date'])


Error adding column 'Game ID': no such table: betting_data_NCAAF
Error adding column 'Game Name_caesars': no such table: betting_data_NCAAF
Error adding column 'Bet Name': no such table: betting_data_NCAAF
Error adding column 'Market Name': no such table: betting_data_NCAAF
Error adding column 'Sportsbook_caesars': no such table: betting_data_NCAAF
Error adding column 'line_caesars': no such table: betting_data_NCAAF
Error adding column 'Odds_caesars': no such table: betting_data_NCAAF
Error adding column 'Player Name_caesars': no such table: betting_data_NCAAF
Error adding column 'Game Name_pinnacle': no such table: betting_data_NCAAF
Error adding column 'Sportsbook_pinnacle': no such table: betting_data_NCAAF
Error adding column 'line_pinnacle': no such table: betting_data_NCAAF
Error adding column 'Odds_pinnacle': no such table: betting_data_NCAAF
Error adding column 'Player Name_pinnacle': no such table: betting_data_NCAAF
Error adding column 'Game Name': no such table: betting_dat

In [90]:
# Connect to the SQLite database
conn = sqlite3.connect('betting_data.db')

# Define the table name
table_name = f'betting_data_{league}'

# Read the data from the SQLite table into a pandas DataFrame
query = f"SELECT * FROM {table_name}"
df = pd.read_sql(query, conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


             Game ID             Game Name_caesars          Bet Name  \
0  15012-77398-24-39          Wyoming vs Air Force        Under 36.5   
1  15175-36978-24-39        Penn State vs Illinois          Illinois   
2  35985-75298-24-39   Memphis vs Middle Tennessee  Middle Tennessee   
3  13936-14627-24-39  Michigan State vs Ohio State    Michigan State   

    Market Name Sportsbook_caesars  line_caesars  Odds_caesars  \
0  Total Points            Caesars          36.5        -110.0   
1     Moneyline            Caesars           NaN         650.0   
2     Moneyline            Caesars           NaN        1550.0   
3     Moneyline            Caesars           NaN        1350.0   

  Player Name_caesars            Game Name_pinnacle Sportsbook_pinnacle  ...  \
0                None          Wyoming vs Air Force            Pinnacle  ...   
1                None        Penn State vs Illinois            Pinnacle  ...   
2                None   Memphis vs Middle Tennessee            Pinna

In [91]:
fdf = (final_ev_df[['Game ID','Game Name_caesars','Bet Name','Market Name','line_caesars','Odds_caesars','Odds_pinnacle','EV_caesars']]).sort_values('EV_caesars',ascending=False)

fdf[abs(fdf['Odds_caesars'] < 1000)]